# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,taungoo,18.9333,96.4333,22.41,87,56,1.37,MM,1701650637
1,1,portland,45.5234,-122.6762,14.69,94,100,2.68,US,1701649950
2,2,grytviken,-54.2811,-36.5092,0.77,73,8,1.96,GS,1701650637
3,3,lebu,-37.6167,-73.6500,11.42,80,0,10.81,CL,1701650637
4,4,xining,36.6167,101.7667,-1.86,39,0,1.10,CN,1701650637


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    size = "Humidity",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_condition =city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0)] 
# Drop any rows with null values
ideal_condition = ideal_condition.dropna()

# Display sample data
ideal_condition.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,prieska,-29.6641,22.7474,21.22,10,0,1.15,ZA,1701650637
45,45,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.51,GF,1701650642
110,110,taoudenni,22.6783,-3.9836,22.97,37,0,3.09,ML,1701650652
206,206,cabo san lucas,22.8909,-109.9124,26.17,52,0,2.06,MX,1701650666
272,272,fort bragg,35.1390,-79.0060,21.01,87,0,4.12,US,1701650675


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
City= ideal_condition['City']
Country= ideal_condition['Country']
Lat= ideal_condition['Lat']
Lng= ideal_condition['Lng']
Humidity= ideal_condition['Humidity']

hotel_df = pd.DataFrame({
                "City": City,
                "Country": Country,
                "Lat": Lat,
                "Lng": Lng, 
                "Humidity": Humidity
                    })

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,prieska,ZA,-29.6641,22.7474,10,
45,remire-montjoly,GF,4.9167,-52.2667,94,
110,taoudenni,ML,22.6783,-3.9836,37,
206,cabo san lucas,MX,22.8909,-109.9124,52,
272,fort bragg,US,35.1390,-79.0060,87,
310,port lincoln,AU,-34.7333,135.8667,45,
356,chicomuselo,MX,15.7667,-92.2667,70,
486,cape coast,GH,5.1053,-1.2466,86,
588,tabou,CI,4.4230,-7.3528,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = { 
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    #params["categories"] = f"accommodation.hotel.{Country}" 
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
prieska - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
taoudenni - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
fort bragg - nearest hotel: Airborne Inn Lodging
port lincoln - nearest hotel: Boston Hotel
chicomuselo - nearest hotel: No hotel found
cape coast - nearest hotel: Mighty Victory Hotel
tabou - nearest hotel: hôtel le rochet


,City,Country,Lat,Lng,Humidity,Hotel Name
5,prieska,ZA,-29.6641,22.7474,10,No hotel found
45,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
110,taoudenni,ML,22.6783,-3.9836,37,No hotel found
206,cabo san lucas,MX,22.8909,-109.9124,52,Comfort Rooms
272,fort bragg,US,35.1390,-79.0060,87,Airborne Inn Lodging
310,port lincoln,AU,-34.7333,135.8667,45,Boston Hotel
356,chicomuselo,MX,15.7667,-92.2667,70,No hotel found
486,cape coast,GH,5.1053,-1.2466,86,Mighty Victory Hotel
588,tabou,CI,4.4230,-7.3528,85,hôtel le rochet


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)